Populate mysql database, compounds, with pubchem cid's

In [3]:
%load_ext dotenv
%dotenv
import pymysql as pymysql
import pubchempy as pcp
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.unichem import unichem_client as unichem
from chemspipy import ChemSpider

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Open connection to gcloud database

In [4]:
try:
    db = pymysql.connect(host='35.203.151.202', user='hts', password='driver', db='compounds',autocommit=True,
                                                     cursorclass=pymysql.cursors.DictCursor)
    print('Connected to remote mysql database:',db.get_host_info())
except Exception as e:
    print("Unable to connect to mysql database -- skipping db entry: ",e)

Connected to remote mysql database: socket 35.203.151.202:3306


In [100]:
cs = ChemSpider('dltdyHyYJnU38iIXsApm05cHVezcmj8V')
res=cs.search('WRJHSKFDDFMOLR-FBCYGCLPSA-N')
print(f"Got {len(res)} results")
for c1 in res:
    print(c1.monoisotopic_mass)

Got 0 results


Verify that monoisotopic masses are the same as in PubChem

In [ ]:
db.close()

In [5]:
with db.cursor() as cursor, db.cursor() as cursor2:
    cursor.execute(f"select * from compounds where compound not in (select n.compound from names n where n.domain=(select d.domain from domains d where d.name='PubChem.cid'))")
    total=0
    while True:
        res=cursor.fetchmany(100)
        if len(res) == 0:
            break
        allkeys=[x['inChIKey'] for x in res]
        pc = pcp.get_compounds(allkeys, 'inchikey')
        for i in range(len(res)):
            row=res[i]
            rec=[x for x in pc if x.inchikey == row['inChIKey'] ]
            #assert len(r)==1
            #print(f"row={row}, rec={rec}")
            if row['monoisotopicMass'] is None:
                print(f"Missing monoisotopic mass for {row['name']} in db. PubChem has {rec[0].monoisotopic_mass}")
                continue
            if len(rec)==0:
                print(f"No PubChem record for {row['name']} {row['inChIKey']}")
                try:
                    unires = unichem.inchiFromKey(row['inChIKey'])
                except:
                    print("Failed UniChem lookup of InChIKey")
                    continue
                #print('UniChem: ',unires)
                inchi=unires[0]['standardinchi']
                try:
                    rec=pcp.get_compounds(inchi,'inchi')
                except:
                    print("Failed pubchem lookup of inchi")
                    continue
                if len(rec)==0:
                    print("Still not found in PubChem")
                    try:
                        molecule = new_client.molecule
                        molecule.get(row['inChIKey'])
                        print(f"Found in chemspider")
                    except:
                        print('Failed retrieve from Chembl')
                    continue;
                else:
                    print(f"Found {len(rec)} using InChI: new key: {rec[0].inchikey}")

            if len(rec)>1:
                print(f"Multiple ({len(rec)}) PubChem record for {row['name']} ({row['inChIKey']})")
            r=rec[0]
            #assert row['inChIKey']==r.inchikey

            if r.monoisotopic_mass is None:
                print(f"PubChem record for {row['name']} has no monoisotopic mass")
            else:
                if abs(row['monoisotopicMass']-r.monoisotopic_mass)>1e-5:
                    print(f"{row['name']}: PubChem: {r.monoisotopic_mass} != db: {row['monoisotopicMass']}")
                elif row['formula'] != r.molecular_formula:
                      print(f"{row['name']}: PubChem: {r.molecular_formula} != db: {row['formula']}")
                else:
                    # Found a record and its mass matched, add to DB
                    nins=cursor2.execute(f"insert into names(compound,domain,name) select {row['compound']},domain,{r.cid} FROM domains d WHERE d.name='PubChem.cid'")
                    print(f"Inserted {nins} PubChem cids")

        total+=len(res)
        print(f"Verified {total} compounds.")

No PubChem record for 45B02 CHLUEHQJBYCJFB-ZBJSNUHESA-N
Failed UniChem lookup of InChIKey
No PubChem record for 206E05 HXKPAOIVNDZXRV-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 208H04 IHUZQHMVGRMOMY-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 247B08 DHQPXEWFKUFBNI-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 247C06 CQDSZTTZSMUKHJ-VMPITWQZSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 247E05 SRCWXTCNSWQWQW-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 286E11 VDWNKHDMRWRPSD-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 287E04 AWNUEWCURVJWFW-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 326C10 YTTIZRYNSCITDP-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 445F10 ATRNLYRMZZLRQX-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey
No PubChem record for 446A08 UHORFKGHEGMECC-UHFFFAOYSA-N
Failed UniChem lookup of InChIKey


In [89]:
#molecule = new_client.molecule
molecule.get('BSYNRYMUTXBXSQ-UHFFFAOYSA-N')
#molecule.get('WRJHSKFDDFMOLR-FBCYGCLPSA-N')

Multiple (2) PubChem record for Water (XLYOFNOQVPJJNP-UHFFFAOYSA-N)
Verified 100 compounds.
No PubChem record for 7A06 WRJHSKFDDFMOLR-FBCYGCLPSA-N


RetryError: HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/WRJHSKFDDFMOLR-FBCYGCLPSA-N (Caused by ResponseError('too many 404 error responses'))

In [6]:
r1 = unichem.inchiFromKey('WRJHSKFDDFMOLR-FBCYGCLPSA-N')
r2 = unichem.inchiFromKey('BSYNRYMUTXBXSQ-UHFFFAOYSA-N')
print(r1,r2)

[{'standardinchi': 'InChI=1S/C16H11N7O/c1-10-11(6-17)15-21-13-4-2-3-5-14(13)23(15)16(24)12(10)7-19-22-9-18-8-20-22/h2-5,7-9,24H,1H3/b19-7+'}] [{'standardinchi': 'InChI=1S/C9H8O4/c1-6(10)13-8-5-3-2-4-7(8)9(11)12/h2-5H,1H3,(H,11,12)'}]


In [27]:
with db.cursor() as cursor:
    cursor.execute(f"select * from compounds")
    for row in cursor:
        rec = pcp.get_compounds(row['inChIKey'], 'inchikey')
        for r in rec:
            if r.monoisotopic_mass is not None and row['monoisotopicMass'] is not None:
                if abs(row['monoisotopicMass']-r.monoisotopic_mass)>1e-6:
                    print(f"{row['name']}: {r.monoisotopic_mass} != {row['monoisotopicMass']}")
                    break

5A02: 396.100748 != 396.101
5A03: 221.0243562 != 221.024
5A04: 398.14778644 != 398.148
5A05: 324.01095 != 324.011
5A06: 329.16270822 != 329.163
5A07: 419.07322 != 419.073
5A08: 356.0694478 != 356.069
5A09: 400.14567844 != 400.146
5A10: 459.20457304 != 459.205
5A11: 411.20457304 != 411.205
5B02: 328.17869264 != 328.179
5B03: 382.15287182 != 382.153
5B04: 322.09872824 != 322.099
5B05: 394.18925733 != 394.189
5B06: 355.17835829 != 355.178
5B07: 412.1634365 != 412.163
5B08: 369.19400835 != 369.194
5B09: 262.09535694 != 262.095
5B10: 357.15762284 != 357.158
5B11: 399.20457304 != 399.205
5C02: 369.19400835 != 369.194
5C03: 390.0537978 != 390.054
5C04: 420.0643625 != 420.064
5C05: 262.07759887 != 262.078
5C06: 335.0924358 != 335.092
5C07: 442.18925733 != 442.189
5C08: 304.14230713 != 304.142
5C09: 377.16270822 != 377.163
5C10: 342.19434271 != 342.194


KeyboardInterrupt: 